# Pulitzer Insights

Q1. Newspaper with the maximum most number of Pulitzer prices?
Q2. What are the top 5 states?
Q3. To find if there is any Correlation between Crime, GDP, and Population on Pulitzer? For example - higher GDP means more prices (confounding parameter could be more journalists) or crime-prone cities incubate investigative journalism resulting in more Pulitzer.
Q4. If there is any correlation between daily circulation and crime rate? 
Q5. To find socioeconomic factors affecting Pulitzer. 

Reading data file from GCP.

In [92]:
import pandas as pd
from google.cloud import storage

import datetime as dt
from datetime import datetime
from pytz import timezone

import uuid

#Reading Google Buckets for files
client = storage.Client()
bucket=client.get_bucket('capstone_project_sr')
blob = storage.Blob('pulitzer.csv',bucket)
with open('pulitzer.csv', 'wb') as file_obj:
    blob.download_to_file(file_obj)
df=pd.read_csv('pulitzer.csv',sep=',',header=0, \
               names=['Newspaper','state','DailyCirculation_2004',\
               'DailyCirculation_2013',\
               'ChangeInDailyCirculation_2004_2013',\
               'WinnersAndFinalists_1990_2003',\
               'WinnersAndFinalists_2004_2014',\
               'WinnersAndFinalists_1990_2014'])

Adding basic data audit fields just in case we run into conflicts later.

In [93]:
tz = timezone('EST') # adding time zone info
datetime.now(tz) 
df['Entrydate'] = dt.datetime.now()

df.insert(0,'Id',uuid.uuid4()) 
df.Id= df.Id.apply(lambda x: uuid.uuid4()) # adding unique identifier

In [94]:
df.info() # checking the data frame structure

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 10 columns):
Id                                    50 non-null object
Newspaper                             50 non-null object
state                                 50 non-null object
DailyCirculation_2004                 50 non-null object
DailyCirculation_2013                 50 non-null object
ChangeInDailyCirculation_2004_2013    50 non-null object
WinnersAndFinalists_1990_2003         50 non-null int64
WinnersAndFinalists_2004_2014         50 non-null int64
WinnersAndFinalists_1990_2014         50 non-null int64
Entrydate                             50 non-null datetime64[ns]
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 4.0+ KB


In [95]:
df

,Id,Newspaper,state,DailyCirculation_2004,DailyCirculation_2013,ChangeInDailyCirculation_2004_2013,WinnersAndFinalists_1990_2003,WinnersAndFinalists_2004_2014,WinnersAndFinalists_1990_2014,Entrydate
0,65607d2b-8e0b-4584-8bd8-892b1c13ed46,USA Today,Virginia,"2,192,098","1,674,306",-24%,1,1,2,2017-10-30 02:53:34.858782
1,dd478f7b-b8a5-4bd2-9ea9-094b0d4e2a4d,Wall Street Journal,New York,"2,101,017","2,378,827",13%,30,20,50,2017-10-30 02:53:34.858782
2,97d76d96-2121-487d-8ecc-ff8b911a4ac9,New York Times,New York,"1,119,027","1,865,318",67%,55,62,117,2017-10-30 02:53:34.858782
3,90940f2b-0524-469b-b782-8166021f6861,Los Angeles Times,California,"983,727","653,868",-34%,44,41,85,2017-10-30 02:53:34.858782
4,a2b429c7-7588-4c1f-b49e-85c1edd757ea,Washington Post,District of Columbia,"760,034","474,767",-38%,52,48,100,2017-10-30 02:53:34.858782
5,6e89e14d-3e96-4c93-8245-f33c19870a23,New York Daily News,New York,"712,671","516,165",-28%,4,2,6,2017-10-30 02:53:34.858782
6,d2b19207-75b4-430a-b00b-db809f0d7f7e,New York Post,New York,"642,844","500,521",-22%,0,0,0,2017-10-30 02:53:34.858782
7,304050f6-3899-487e-8327-d8ac40062b00,Chicago Tribune,Illinois,"603,315","414,930",-31%,23,15,38,2017-10-30 02:53:34.858782
8,93c27b54-e290-4428-b201-75ad586e6f57,San Jose Mercury News,California,"558,874","583,998",4%,4,2,6,2017-10-30 02:53:34.858782
9,e868e9db-783e-4f3d-be31-50dcce24b273,Newsday,New York,"553,117","377,744",-32%,12,6,18,2017-10-30 02:53:34.858782


### Inserting data into Cassandra database for persistence and reliability among other benefits.

Reading the connection points details.

In [80]:
df_con=pd.read_csv('~/connection_point.csv',header=0)

In [96]:
import itertools
from multiprocessing import Pool
import sys
import time
from cassandra.cluster import Cluster
from cassandra.concurrent import execute_concurrent_with_args
from cassandra.query import tuple_factory
from cassandra.auth import PlainTextAuthProvider

def _insertData(params):
    cluster = Cluster(contact_points=[df_con.ip[0]], auth_provider = \
                      PlainTextAuthProvider(username=df_con.user[0], \
                                            password=df_con.token[0]))
    session = cluster.connect()
    session.set_keyspace('capstone')
    session.row_factory = tuple_factory
    prepared=session.prepare("INSERT INTO capstone.pulitzer \
                             (id,Newspaper,state,DailyCirculation_2004,DailyCirculation_2013, \
                             ChangeDailyCirculation_2004_2013,WinNFinalists_1990_2003, \
                             WinNFinalists_2004_2014,WinNFinalists_1990_2014,Entrydate) \
                             VALUES (?,?,?,?,?,?,?,?,?,?)")
    
    #using datastax driver for multiprocessing 
    execute_concurrent_with_args(session, prepared, params, concurrency=50) 
    return None

def multiprocess(params):
    pool = Pool(processes=4)
    results = [pool.map(_insertData, (params[n:n+100],)) for n in range(0, len(params),100)]
    return results
    

if __name__ == "__main__":
    parameters=[]
    for index, row in enumerate(df.values):        
        (a,b,c,d,e,f,g,h,i,j) = row
        row1=(a,str(b),str(c),str(d),str(e),str(f),str(g),str(h),str(i),j)
        parameters.append(row1)           
    a = multiprocess(parameters)

### Reading GDP by state and contry

In [71]:
import os

In [73]:
files = [file for file in os.listdir( './Data' ) \
         if file.startswith("usgs_1957_2015")]
gdp_merged=pd.DataFrame()

for file_ in files:
    filename='./Data/'+file_
    df = pd.read_csv(filename,skiprows=0,header=1,skipfooter=6,error_bad_lines=False, engine='python', usecols=range(0,3))
    df['State']=df.columns[2].split('-')[1][:2]
    del df[df.columns[2]]
    df.columns=['year','GDP-billion','state']
    gdp_merged=pd.concat([gdp_merged,df],ignore_index=1,axis=0)

Skipping line 67: ',' expected after '"'. Error could possibly be due to parsing errors in the skipped footer rows (the skipfooter keyword is only applied after Python's csv library has parsed all rows).
Skipping line 68: ',' expected after '"'. Error could possibly be due to parsing errors in the skipped footer rows (the skipfooter keyword is only applied after Python's csv library has parsed all rows).
Skipping line 69: ',' expected after '"'. Error could possibly be due to parsing errors in the skipped footer rows (the skipfooter keyword is only applied after Python's csv library has parsed all rows).
Skipping line 72: ',' expected after '"'. Error could possibly be due to parsing errors in the skipped footer rows (the skipfooter keyword is only applied after Python's csv library has parsed all rows).
Skipping line 73: ',' expected after '"'. Error could possibly be due to parsing errors in the skipped footer rows (the skipfooter keyword is only applied after Python's csv library ha

Skipping line 67: ',' expected after '"'. Error could possibly be due to parsing errors in the skipped footer rows (the skipfooter keyword is only applied after Python's csv library has parsed all rows).
Skipping line 68: ',' expected after '"'. Error could possibly be due to parsing errors in the skipped footer rows (the skipfooter keyword is only applied after Python's csv library has parsed all rows).
Skipping line 69: ',' expected after '"'. Error could possibly be due to parsing errors in the skipped footer rows (the skipfooter keyword is only applied after Python's csv library has parsed all rows).
Skipping line 72: ',' expected after '"'. Error could possibly be due to parsing errors in the skipped footer rows (the skipfooter keyword is only applied after Python's csv library has parsed all rows).
Skipping line 73: ',' expected after '"'. Error could possibly be due to parsing errors in the skipped footer rows (the skipfooter keyword is only applied after Python's csv library ha

Skipping line 73: ',' expected after '"'. Error could possibly be due to parsing errors in the skipped footer rows (the skipfooter keyword is only applied after Python's csv library has parsed all rows).
Skipping line 74: ',' expected after '"'. Error could possibly be due to parsing errors in the skipped footer rows (the skipfooter keyword is only applied after Python's csv library has parsed all rows).
Skipping line 75: ',' expected after '"'. Error could possibly be due to parsing errors in the skipped footer rows (the skipfooter keyword is only applied after Python's csv library has parsed all rows).
Skipping line 67: ',' expected after '"'. Error could possibly be due to parsing errors in the skipped footer rows (the skipfooter keyword is only applied after Python's csv library has parsed all rows).
Skipping line 68: ',' expected after '"'. Error could possibly be due to parsing errors in the skipped footer rows (the skipfooter keyword is only applied after Python's csv library ha

In [74]:
tz = timezone('EST') # adding time zone info
datetime.now(tz) 
gdp_merged['Entrydate'] = dt.datetime.now()

gdp_merged.insert(0,'Id',uuid.uuid4()) 
gdp_merged.Id= gdp_merged.Id.apply(lambda x: uuid.uuid4()) # adding unique identifier

In [75]:
def _insertData(params):
    cluster = Cluster(contact_points=[df_con.ip[0]], auth_provider = \
                      PlainTextAuthProvider(username=df_con.user[0], \
                                            password=df_con.token[0]))
    session = cluster.connect()
    session.set_keyspace('capstone')
    session.row_factory = tuple_factory
    prepared=session.prepare("INSERT INTO capstone.GDP(id,year,GDP,state,entrydate) VALUES (?,?,?,?,?)")
    
    #using datastax driver for multiprocessing 
    execute_concurrent_with_args(session, prepared, params, concurrency=50) 
    return None

def multiprocess(params):
    pool = Pool(processes=4)
    results = [pool.map(_insertData, (params[n:n+100],)) for n in range(0, len(params),100)]
    return results
    

if __name__ == "__main__":
    parameters=[]
    for index, row in enumerate(gdp_merged.values):        
        (a,b,c,d,e) = row
        row1=(a,str(b),int(c),str(d),i)
        parameters.append(row1)           
    a = multiprocess(parameters)

In [44]:
blob = storage.Blob('CrimeIndex.xlsx',bucket)
with open('Crimeindex.xlsx', 'wb') as file_obj:
    blob.download_to_file(file_obj)
    
xl=pd.ExcelFile('Crimeindex.xlsx')
df_crime=xl.parse('Sheet1',header=0)

In [46]:
tz = timezone('EST') # adding time zone info
datetime.now(tz) 
df_crime['Entrydate'] = dt.datetime.now()

df_crime.insert(0,'Id',uuid.uuid4()) 
df_crime.Id= df_crime.Id.apply(lambda x: uuid.uuid4()) # adding unique identifier

In [57]:
df_crime.columns=['Id','rank','crimeindex','state-population','Entrydate']

In [83]:
df_crime.insert(4,'state','')
df_crime['state']=df_crime['state-population'].apply(lambda x: x.split('/')[0][:-1])

In [85]:
df_crime.insert(5,'population','')
df_crime['population']=df_crime['state-population'].apply(lambda x: x.split('/')[1].strip())

In [88]:
del df_crime[df_crime.columns[3]]

In [89]:
df_crime#['state-population'].values[0].split('/')[1].strip()

,Id,rank,crimeindex,state,population,Entrydate
0,7fa8d8bc-6df4-474b-897c-bad1880e3b2e,1,950,Vermont,"626,358",2017-10-22 23:05:01.480357
1,e9599ed6-8805-4f31-a07b-2149b6690b4a,2,956,New Hampshire,"1,321,069",2017-10-22 23:05:01.480357
2,669e8915-a451-49f3-8b0a-9b8202256ae5,3,985,North Dakota,"704,925",2017-10-22 23:05:01.480357
3,d2919cd0-337c-4144-bdc7-deef18b5789b,4,1017,Maine,"1,328,535",2017-10-22 23:05:01.480357
4,b1909949-f893-4257-85f1-4d284245d826,5,1042,South Dakota,"834,708",2017-10-22 23:05:01.480357
5,ebd33c0e-8506-49df-a5a3-d62e9c9061bf,6,1068,Idaho,"1,599,464",2017-10-22 23:05:01.480357
6,01788308-b9e4-4be9-83ca-7005561d114b,7,1125,Virginia,"8,185,131",2017-10-22 23:05:01.480357
7,a98805b2-bc79-48fe-985b-0138cf4c9a36,8,1136,Wyoming,"575,251",2017-10-22 23:05:01.480357
8,b774e183-9c16-45a9-80fb-78e00c59c17a,9,1171,Connecticut,"3,592,053",2017-10-22 23:05:01.480357
9,f0f7cc57-abfc-410d-8df6-5ec48b42ebdf,10,1180,New Jersey,"8,874,374",2017-10-22 23:05:01.480357


In [100]:
def _insertData(params):
    cluster = Cluster(contact_points=[df_con.ip[0]], auth_provider = \
                      PlainTextAuthProvider(username=df_con.user[0], \
                                            password=df_con.token[0]))
    session = cluster.connect()
    session.set_keyspace('capstone')
    session.row_factory = tuple_factory
    prepared=session.prepare("INSERT INTO capstone.crimeindex(id,rank,crimeindex,state,population,entrydate)\
                              VALUES (?,?,?,?,?,?)")
    
    #using datastax driver for multiprocessing 
    execute_concurrent_with_args(session, prepared, params, concurrency=50) 
    return None

def multiprocess(params):
    pool = Pool(processes=4)
    results = [pool.map(_insertData, (params[n:n+100],)) for n in range(0, len(params),100)]
    return results
    

if __name__ == "__main__":
    parameters=[]
    for index, row in enumerate(df_crime.values):        
        (a,b,c,d,e,f) = row
        row1=(a,int(b),int(c),str(d),int(e.replace(',','')),f)
        parameters.append(row1)           
    a = multiprocess(parameters)

In [103]:
blob = storage.Blob('table_1_crime_in_the_united_states_by_volume_and_rate_per_100000_inhabitants_1995-2014.xls',bucket)
with open('table_1_crime_in_the_united_states_by_volume_and_rate_per_100000_inhabitants_1995-2014.xls', 'wb') as file_obj:
    blob.download_to_file(file_obj)

In [104]:
filename='table_1_crime_in_the_united_states_by_volume_and_rate_per_100000_inhabitants_1995-2014.xls'
xl=pd.ExcelFile(filename)
df_crime_byvol=xl.parse('14tbl01',skiprows=3,header=0,skipfooter=10,error_bad_lines=False, engine='python', usecols=range(0,4))

In [106]:
df_crime_byvol['Year']=df_crime_byvol.Year.apply(lambda x: int(str(x)[:4]))

In [108]:
df_crime_byvol.columns=['year','population','violentcrime','violentcrimerate']

In [110]:
tz = timezone('EST') # adding time zone info
datetime.now(tz) 
df_crime_byvol['Entrydate'] = dt.datetime.now()

df_crime_byvol.insert(0,'Id',uuid.uuid4()) 
df_crime_byvol.Id= df_crime.Id.apply(lambda x: uuid.uuid4()) # adding unique identifier

In [111]:
df_crime_byvol

,Id,year,population,violentcrime,violentcrimerate,Entrydate
0,921006ea-e4ad-412f-b0c8-9a494feb08fb,1995,262803276,1798792,684.5,2017-10-23 00:27:37.706506
1,3cda340b-794f-49da-9b73-82fdd189c67d,1996,265228572,1688540,636.6,2017-10-23 00:27:37.706506
2,8d58a704-0aa5-46a8-9868-e581592bffc1,1997,267783607,1636096,611.0,2017-10-23 00:27:37.706506
3,bfac0bd3-8fc6-4cf0-b5bd-16a9a35117fa,1998,270248003,1533887,567.6,2017-10-23 00:27:37.706506
4,80be7628-bd41-44c6-87fb-310667d546cc,1999,272690813,1426044,523.0,2017-10-23 00:27:37.706506
5,69f9c891-aa66-40e3-81d7-b2cd728418be,2000,281421906,1425486,506.5,2017-10-23 00:27:37.706506
6,e1fd6370-291f-44e4-b742-5b34689a5d87,2001,285317559,1439480,504.5,2017-10-23 00:27:37.706506
7,c5030888-1574-49e9-baea-48410b12a52d,2002,287973924,1423677,494.4,2017-10-23 00:27:37.706506
8,edce9436-225c-4d11-99b9-ca8d85bbfcfc,2003,290788976,1383676,475.8,2017-10-23 00:27:37.706506
9,1b4b77c0-7642-4125-869a-d2e6b5ba52a9,2004,293656842,1360088,463.2,2017-10-23 00:27:37.706506


In [112]:
def _insertData(params):
    cluster = Cluster(contact_points=[df_con.ip[0]], auth_provider = \
                      PlainTextAuthProvider(username=df_con.user[0], \
                                            password=df_con.token[0]))
    session = cluster.connect()
    session.set_keyspace('capstone')
    session.row_factory = tuple_factory
    prepared=session.prepare("INSERT INTO capstone.crimebyvol(id,year,population, \
                              violentcrime,violentcrimerate,entrydate) \
                              VALUES (?,?,?,?,?,?)")
    
    #using datastax driver for multiprocessing 
    execute_concurrent_with_args(session, prepared, params, concurrency=50) 
    return None

def multiprocess(params):
    pool = Pool(processes=4)
    results = [pool.map(_insertData, (params[n:n+100],)) for n in range(0, len(params),100)]
    return results
    
if __name__ == "__main__":
    parameters=[]
    for index, row in enumerate(df_crime_byvol.values):        
        (a,b,c,d,e,f) = row
        row1=(a,str(b),int(c),int(d),int(e),f)
        parameters.append(row1)           
    a = multiprocess(parameters)

In [14]:
import pandas as pd
filename='./Data/st-est00int-alldata.csv'
df_=pd.read_csv(filename,header=0,usecols=[2,3,4,5,6,7,9,10,11,12,13,14,15,16,17,18])

In [15]:
df_.columns

Index(['STATE', 'NAME', 'SEX', 'ORIGIN', 'RACE', 'AGEGRP', 'POPESTIMATE2000',
       'POPESTIMATE2001', 'POPESTIMATE2002', 'POPESTIMATE2003',
       'POPESTIMATE2004', 'POPESTIMATE2005', 'POPESTIMATE2006',
       'POPESTIMATE2007', 'POPESTIMATE2008', 'POPESTIMATE2009'],
      dtype='object')

In [18]:
df_pop=df_[(df_.SEX==0) & (df_.ORIGIN==0) & (df_.RACE==0) & (df_.AGEGRP==0)]

In [19]:
df_=df_pop.drop(['SEX', 'ORIGIN', 'RACE', 'AGEGRP'], axis=1)

In [20]:
df_.columns

Index(['STATE', 'NAME', 'POPESTIMATE2000', 'POPESTIMATE2001',
       'POPESTIMATE2002', 'POPESTIMATE2003', 'POPESTIMATE2004',
       'POPESTIMATE2005', 'POPESTIMATE2006', 'POPESTIMATE2007',
       'POPESTIMATE2008', 'POPESTIMATE2009'],
      dtype='object')

In [16]:
filename='./Data/nst-est2016-alldata.csv'
df_1=pd.read_csv(filename,header=0,usecols=[1,3,4,7,8,9,10,11,12,13])

In [17]:
df_1.columns

Index(['REGION', 'STATE', 'NAME', 'POPESTIMATE2010', 'POPESTIMATE2011',
       'POPESTIMATE2012', 'POPESTIMATE2013', 'POPESTIMATE2014',
       'POPESTIMATE2015', 'POPESTIMATE2016'],
      dtype='object')

In [21]:
df_pop_merged=pd.merge(df_1,df_,how='left', on='NAME')

In [26]:
df_pop_merged.columns#sample(15,axis=1)#.info()

Index(['REGION', 'STATE_x', 'NAME', 'POPESTIMATE2010', 'POPESTIMATE2011',
       'POPESTIMATE2012', 'POPESTIMATE2013', 'POPESTIMATE2014',
       'POPESTIMATE2015', 'POPESTIMATE2016', 'STATE_y', 'POPESTIMATE2000',
       'POPESTIMATE2001', 'POPESTIMATE2002', 'POPESTIMATE2003',
       'POPESTIMATE2004', 'POPESTIMATE2005', 'POPESTIMATE2006',
       'POPESTIMATE2007', 'POPESTIMATE2008', 'POPESTIMATE2009'],
      dtype='object')

In [27]:
df_pop_merged_=df_pop_merged.drop(['STATE_x', 'STATE_y'], axis=1)

In [31]:
tz = timezone('EST') # adding time zone info
datetime.now(tz) 
df_pop_merged_['Entrydate'] = dt.datetime.now()

df_pop_merged_.insert(0,'Id',uuid.uuid4()) 
df_pop_merged_.Id= df_pop_merged_.Id.apply(lambda x: uuid.uuid4()) # adding unique identifier

In [41]:
df_pop_merged_.sample(1DC0)#.columns

,Id,REGION,NAME,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015,POPESTIMATE2016,...,POPESTIMATE2001,POPESTIMATE2002,POPESTIMATE2003,POPESTIMATE2004,POPESTIMATE2005,POPESTIMATE2006,POPESTIMATE2007,POPESTIMATE2008,POPESTIMATE2009,Entrydate
6,ef28f59e-ee8d-4114-881d-dc0bea54300d,4,Alaska,714031,722713,731089,736879,736705,737709,741894,...,633714.0,642337.0,648414.0,659286.0,666946.0,675302.0,680300.0,687455.0,698895.0,2017-10-29 00:37:14.485454
27,a45d1278-4ff5-4c4e-87d5-b07380248d1d,2,Michigan,9877495,9876213,9887238,9898982,9915767,9917715,9928300,...,9991120.0,10015710.0,10041152.0,10055315.0,10051137.0,10036081.0,10001284.0,9946889.0,9901591.0,2017-10-29 00:37:14.485454
24,02f24633-501e-4beb-9944-9b42a85bc2c0,1,Maine,1327730,1328231,1328895,1329076,1330719,1329453,1331479,...,1285692.0,1295960.0,1306513.0,1313688.0,1318787.0,1323619.0,1327040.0,1330509.0,1329590.0,2017-10-29 00:37:14.485454
9,23d47ec3-692d-418e-8384-4feae6461964,4,California,37332685,37676861,38011074,38335203,38680810,38993940,39250017,...,34479458.0,34871843.0,35253159.0,35574576.0,35827943.0,36021202.0,36250311.0,36604337.0,36961229.0,2017-10-29 00:37:14.485454
51,89720e3b-a153-4352-80d0-8dd8d439ffe0,3,Virginia,8025773,8110035,8192048,8262692,8317372,8367587,8411808,...,7198362.0,7286873.0,7366977.0,7475575.0,7577105.0,7673725.0,7751000.0,7833496.0,7925937.0,2017-10-29 00:37:14.485454
23,ffa79e72-1e34-4430-8a1f-f0080a37c9af,3,Louisiana,4544996,4575404,4603429,4626402,4647880,4668960,4681666,...,4477875.0,4497267.0,4521042.0,4552238.0,4576628.0,4302665.0,4375581.0,4435586.0,4491648.0,2017-10-29 00:37:14.485454
22,b485db58-be8d-40a0-a726-daeb3e794582,3,Kentucky,4348662,4369354,4384799,4400477,4413057,4424611,4436974,...,4068132.0,4089875.0,4117170.0,4146101.0,4182742.0,4219239.0,4256672.0,4289878.0,4317074.0,2017-10-29 00:37:14.485454
18,cc2a556d-f815-4238-9616-ce8e4344a6ef,2,Illinois,12841578,12860012,12870798,12879505,12867544,12839047,12801539,...,12488445.0,12525556.0,12556006.0,12589773.0,12609903.0,12643955.0,12695866.0,12747038.0,12796778.0,2017-10-29 00:37:14.485454
3,820286e6-bb63-4509-8fac-3df8d7807490,3,South Region,114863114,116061801,117299171,118424320,119696311,121039206,122319574,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-10-29 00:37:14.485454
10,c807068d-193f-402d-ba05-8a8babfc123f,4,Colorado,5048644,5118360,5189867,5267603,5349648,5448819,5540545,...,4425687.0,4490406.0,4528732.0,4575013.0,4631888.0,4720423.0,4803868.0,4889730.0,4972195.0,2017-10-29 00:37:14.485454


In [10]:
import itertools
from multiprocessing import Pool
import sys
import time
from cassandra.cluster import Cluster
from cassandra.concurrent import execute_concurrent_with_args
from cassandra.query import tuple_factory
from cassandra.auth import PlainTextAuthProvider

In [42]:
def _insertData(params):
    cluster = Cluster(contact_points=[df_con.ip[0]], auth_provider = \
                      PlainTextAuthProvider(username=df_con.user[0], \
                                            password=df_con.token[0]))
    session = cluster.connect()
    session.set_keyspace('capstone')
    session.row_factory = tuple_factory
    prepared=session.prepare("INSERT INTO capstone.population(id,region,state,   \
                              POPESTIMATE2000, POPESTIMATE2001, POPESTIMATE2002, \
                              POPESTIMATE2003, POPESTIMATE2004, POPESTIMATE2005, \
                              POPESTIMATE2006, POPESTIMATE2007, POPESTIMATE2008, \
                              POPESTIMATE2009, POPESTIMATE2010, POPESTIMATE2011, \
                              POPESTIMATE2012, POPESTIMATE2013, POPESTIMATE2014, \
                              POPESTIMATE2015, POPESTIMATE2016,entrydate)                  \
                              VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)")
    
    #using datastax driver for multiprocessing 
    execute_concurrent_with_args(session, prepared, params, concurrency=50) 
    return None

def multiprocess(params):
    pool = Pool(processes=4)
    results = [pool.map(_insertData, (params[n:n+100],)) for n in range(0, len(params),100)]
    return results
    
if __name__ == "__main__":
    parameters=[]
    for index, row in enumerate(df_pop_merged_.values):        
        (a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u) = row
        row1=(a,str(b),c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u)
        parameters.append(row1)           
    a = multiprocess(parameters)
    

# Reading data from Cassandra Tables

In [83]:
import itertools
from multiprocessing import Pool
import sys
import time
from cassandra.cluster import Cluster
from cassandra.concurrent import execute_concurrent_with_args
from cassandra.query import tuple_factory
from cassandra.auth import PlainTextAuthProvider

In [110]:
df_code=pd.read_csv('~/states_code.txt',header=0,sep='\t')

In [113]:
states_dict=dict(zip(df_code.Abbreviation,df_code.State))

In [114]:
from IPython.display import display
def pandas_factory(colnames, rows):
    return pd.DataFrame(rows, columns=colnames)

def _fetchData(query_):
    cluster = Cluster(contact_points=[df_con.ip[0]], auth_provider = \
                      PlainTextAuthProvider(username=df_con.user[0], \
                                            password=df_con.token[0]))
    session = cluster.connect()
    session.set_keyspace('capstone')
    session.row_factory = pandas_factory    
    rows = session.execute(query_)
    return rows

if __name__ == "__main__":
    query_pram="SELECT  Newspaper,state,DailyCirculation_2004, \
                DailyCirculation_2013,ChangeDailyCirculation_2004_2013,\
                WinNFinalists_1990_2003,WinNFinalists_2004_2014,WinNFinalists_1990_2014\
                FROM capstone.pulitzer"
    rows = _fetchData(query_pram)
    df_pulitzer=rows._current_rows
    #print(display(df_pulitzer))
    
    #
    query_pram="SELECT  gdp,state,year FROM capstone.gdp"
    rows = _fetchData(query_pram)
    df_gdp=rows._current_rows
    #print(display(df_gdp))  

In [115]:
df_gdp.state=df_gdp.state.apply(lambda x: states_dict[x])

Adding 2004 GDP data

In [116]:
df_gdp_sorted=df_gdp[df_gdp.year=='2004']

In [117]:
df_gdp_sorted.columns=['GDP_2004','state','year']

In [121]:
df__=pd.merge(df_pulitzer,df_gdp_sorted,how='left', on='state')

,newspaper,state,dailycirculation_2004,dailycirculation_2013,changedailycirculation_2004_2013,winnfinalists_1990_2003,winnfinalists_2004_2014,winnfinalists_1990_2014,GDP_2004,year
0,Miami Herald,Florida,"325,032","147,130",-55%,17,7,24,639,2004
1,Los Angeles Times,California,"983,727","653,868",-34%,44,41,85,1645,2004
2,USA Today,Virginia,"2,192,098","1,674,306",-24%,1,1,2,331,2004
3,Kansas City Star,Kansas,"275,747","189,283",-31%,2,0,2,99,2004
4,St. Louis Post-Dispatch,Missouri,"281,198","167,199",-41%,4,3,7,215,2004
5,Newsday,New York,"553,117","377,744",-32%,12,6,18,958,2004
6,Detroit Free Press,Michigan,"379,304","209,652",-45%,7,5,12,382,2004
7,Arizona Republic,Arizona,"466,926","293,640",-37%,5,2,7,204,2004
8,Charlotte Observer,North Carolina,"231,369","137,829",-40%,1,3,4,332,2004
9,Columbus Dispatch,Ohio,"259,127","137,148",-47%,1,0,1,451,2004


Adding 2013 GDP Data

In [125]:
df_gdp_sorted=df_gdp[df_gdp.year=='2014']
df_gdp_sorted.columns=['GDP_2014','state','year']

In [126]:
df__=pd.merge(df__,df_gdp_sorted,how='left', on='state')

In [127]:
df__

,newspaper,state,dailycirculation_2004,dailycirculation_2013,changedailycirculation_2004_2013,winnfinalists_1990_2003,winnfinalists_2004_2014,winnfinalists_1990_2014,GDP_2004,year_x,GDP_2013,year_y,GDP_2014,year
0,Miami Herald,Florida,"325,032","147,130",-55%,17,7,24,639,2004,794,2013,832,2014
1,Los Angeles Times,California,"983,727","653,868",-34%,44,41,85,1645,2004,2223,2013,2349,2014
2,USA Today,Virginia,"2,192,098","1,674,306",-24%,1,1,2,331,2004,449,2013,459,2014
3,Kansas City Star,Kansas,"275,747","189,283",-31%,2,0,2,99,2004,143,2013,147,2014
4,St. Louis Post-Dispatch,Missouri,"281,198","167,199",-41%,4,3,7,215,2004,275,2013,282,2014
5,Newsday,New York,"553,117","377,744",-32%,12,6,18,958,2004,1330,2013,1391,2014
6,Detroit Free Press,Michigan,"379,304","209,652",-45%,7,5,12,382,2004,429,2013,445,2014
7,Arizona Republic,Arizona,"466,926","293,640",-37%,5,2,7,204,2004,270,2013,280,2014
8,Charlotte Observer,North Carolina,"231,369","137,829",-40%,1,3,4,332,2004,454,2013,474,2014
9,Columbus Dispatch,Ohio,"259,127","137,148",-47%,1,0,1,451,2004,564,2013,591,2014


## Adding Crime Data

In [129]:
from IPython.display import display
def pandas_factory(colnames, rows):
    return pd.DataFrame(rows, columns=colnames)

def _fetchData(query_):
    cluster = Cluster(contact_points=[df_con.ip[0]], auth_provider = \
                      PlainTextAuthProvider(username=df_con.user[0], \
                                            password=df_con.token[0]))
    session = cluster.connect()
    session.set_keyspace('capstone')
    session.row_factory = pandas_factory    
    rows = session.execute(query_)
    return rows

if __name__ == "__main__":
    query_pram="SELECT  crimeindex,state FROM capstone.crimeindex"
    rows = _fetchData(query_pram)
    df_crimeindex=rows._current_rows
    #print(display(df_pulitzer))
    
    #
    query_pram="SELECT  violentcrime,year FROM capstone.crimebyvol"
    rows = _fetchData(query_pram)
    df_crimebyvol=rows._current_rows
    #print(display(df_gdp))  

In [131]:
#df_crimeindex
df__=pd.merge(df__,df_crimeindex,how='left', on='state')

In [132]:
df__

,newspaper,state,dailycirculation_2004,dailycirculation_2013,changedailycirculation_2004_2013,winnfinalists_1990_2003,winnfinalists_2004_2014,winnfinalists_1990_2014,GDP_2004,year_x,GDP_2013,year_y,GDP_2014,year,crimeindex
0,Miami Herald,Florida,"325,032","147,130",-55%,17,7,24,639,2004,794,2013,832,2014,2095
1,Los Angeles Times,California,"983,727","653,868",-34%,44,41,85,1645,2004,2223,2013,2349,2014,1701
2,USA Today,Virginia,"2,192,098","1,674,306",-24%,1,1,2,331,2004,449,2013,459,2014,1125
3,Kansas City Star,Kansas,"275,747","189,283",-31%,2,0,2,99,2004,143,2013,147,2014,1670
4,St. Louis Post-Dispatch,Missouri,"281,198","167,199",-41%,4,3,7,215,2004,275,2013,282,2014,1858
5,Newsday,New York,"553,117","377,744",-32%,12,6,18,958,2004,1330,2013,1391,2014,1194
6,Detroit Free Press,Michigan,"379,304","209,652",-45%,7,5,12,382,2004,429,2013,445,2014,1757
7,Arizona Republic,Arizona,"466,926","293,640",-37%,5,2,7,204,2004,270,2013,280,2014,2025
8,Charlotte Observer,North Carolina,"231,369","137,829",-40%,1,3,4,332,2004,454,2013,474,2014,1869
9,Columbus Dispatch,Ohio,"259,127","137,148",-47%,1,0,1,451,2004,564,2013,591,2014,1691


In [168]:
df_crimebyvol_sorted=df_crimebyvol[df_crimebyvol.year=='2014']

In [171]:
df_crimebyvol_sorted.insert(2,'state','US')

In [172]:
df_crimebyvol_sorted.columns=['violentcrime_2014','year','state']

In [173]:
df__=pd.merge(df__,df_crimebyvol_sorted,how='left', on='year')

In [178]:
df__#.columns

,newspaper,state_x,dailycirculation_2004,dailycirculation_2013,changedailycirculation_2004_2013,winnfinalists_1990_2003,winnfinalists_2004_2014,winnfinalists_1990_2014,GDP_2004,year_x,...,year,crimeindex,violentcrime_2004_x,year_vol,violentcrime_2004_y,state_y,violentcrime_2013,state_x,violentcrime_2014,state_y
0,Miami Herald,Florida,"325,032","147,130",-55%,17,7,24,639,2004,...,2014,2095,NaN,NaN,1360088,US,1168298,US,1165383,US
1,Los Angeles Times,California,"983,727","653,868",-34%,44,41,85,1645,2004,...,2014,1701,NaN,NaN,1360088,US,1168298,US,1165383,US
2,USA Today,Virginia,"2,192,098","1,674,306",-24%,1,1,2,331,2004,...,2014,1125,NaN,NaN,1360088,US,1168298,US,1165383,US
3,Kansas City Star,Kansas,"275,747","189,283",-31%,2,0,2,99,2004,...,2014,1670,NaN,NaN,1360088,US,1168298,US,1165383,US
4,St. Louis Post-Dispatch,Missouri,"281,198","167,199",-41%,4,3,7,215,2004,...,2014,1858,1360088.0,NaN,1360088,US,1168298,US,1165383,US
5,Newsday,New York,"553,117","377,744",-32%,12,6,18,958,2004,...,2014,1194,NaN,NaN,1360088,US,1168298,US,1165383,US
6,Detroit Free Press,Michigan,"379,304","209,652",-45%,7,5,12,382,2004,...,2014,1757,NaN,NaN,1360088,US,1168298,US,1165383,US
7,Arizona Republic,Arizona,"466,926","293,640",-37%,5,2,7,204,2004,...,2014,2025,NaN,NaN,1360088,US,1168298,US,1165383,US
8,Charlotte Observer,North Carolina,"231,369","137,829",-40%,1,3,4,332,2004,...,2014,1869,NaN,NaN,1360088,US,1168298,US,1165383,US
9,Columbus Dispatch,Ohio,"259,127","137,148",-47%,1,0,1,451,2004,...,2014,1691,NaN,NaN,1360088,US,1168298,US,1165383,US


In [179]:
df__=df__.drop(['year_y', 'year_x','year','violentcrime_2004_x','year_vol','state_y','state_x'], axis=1)

In [181]:
df__.columns

Index(['newspaper', 'dailycirculation_2004', 'dailycirculation_2013',
       'changedailycirculation_2004_2013', 'winnfinalists_1990_2003',
       'winnfinalists_2004_2014', 'winnfinalists_1990_2014', 'GDP_2004',
       'GDP_2013', 'GDP_2014', 'crimeindex', 'violentcrime_2004_y',
       'violentcrime_2013', 'violentcrime_2014'],
      dtype='object')

In [182]:
df__.columns=['newspaper', 'dailycirculation_2004', 'dailycirculation_2013',
       'changedailycirculation_2004_2013', 'winnfinalists_1990_2003',
       'winnfinalists_2004_2014', 'winnfinalists_1990_2014', 'GDP_2004',
       'GDP_2013', 'GDP_2014', 'crimeindex', 'violentcrime_2004',
       'violentcrime_2013', 'violentcrime_2014']


In [183]:
df__

,newspaper,dailycirculation_2004,dailycirculation_2013,changedailycirculation_2004_2013,winnfinalists_1990_2003,winnfinalists_2004_2014,winnfinalists_1990_2014,GDP_2004,GDP_2013,GDP_2014,crimeindex,violentcrime_2004,violentcrime_2013,violentcrime_2014
0,Miami Herald,"325,032","147,130",-55%,17,7,24,639,794,832,2095,1360088,1168298,1165383
1,Los Angeles Times,"983,727","653,868",-34%,44,41,85,1645,2223,2349,1701,1360088,1168298,1165383
2,USA Today,"2,192,098","1,674,306",-24%,1,1,2,331,449,459,1125,1360088,1168298,1165383
3,Kansas City Star,"275,747","189,283",-31%,2,0,2,99,143,147,1670,1360088,1168298,1165383
4,St. Louis Post-Dispatch,"281,198","167,199",-41%,4,3,7,215,275,282,1858,1360088,1168298,1165383
5,Newsday,"553,117","377,744",-32%,12,6,18,958,1330,1391,1194,1360088,1168298,1165383
6,Detroit Free Press,"379,304","209,652",-45%,7,5,12,382,429,445,1757,1360088,1168298,1165383
7,Arizona Republic,"466,926","293,640",-37%,5,2,7,204,270,280,2025,1360088,1168298,1165383
8,Charlotte Observer,"231,369","137,829",-40%,1,3,4,332,454,474,1869,1360088,1168298,1165383
9,Columbus Dispatch,"259,127","137,148",-47%,1,0,1,451,564,591,1691,1360088,1168298,1165383


## Adding Population data

In [184]:
from IPython.display import display
def pandas_factory(colnames, rows):
    return pd.DataFrame(rows, columns=colnames)

def _fetchData(query_):
    cluster = Cluster(contact_points=[df_con.ip[0]], auth_provider = \
                      PlainTextAuthProvider(username=df_con.user[0], \
                                            password=df_con.token[0]))
    session = cluster.connect()
    session.set_keyspace('capstone')
    session.row_factory = pandas_factory    
    rows = session.execute(query_)
    return rows

if __name__ == "__main__":
    query_pram="SELECT  * FROM capstone.population"
    rows = _fetchData(query_pram)
    df_popu=rows._current_rows
    #print(display(df_pulitzer))


In [191]:
df_popu_sorted=df_popu[['popestimate2004','popestimate2013','popestimate2014','state']]

In [194]:

df__#=pd.merge(df__,df_popu_sorted,how='left', on='state')

,newspaper,dailycirculation_2004,dailycirculation_2013,changedailycirculation_2004_2013,winnfinalists_1990_2003,winnfinalists_2004_2014,winnfinalists_1990_2014,GDP_2004,GDP_2013,GDP_2014,crimeindex,violentcrime_2004,violentcrime_2013,violentcrime_2014
0,Miami Herald,"325,032","147,130",-55%,17,7,24,639,794,832,2095,1360088,1168298,1165383
1,Los Angeles Times,"983,727","653,868",-34%,44,41,85,1645,2223,2349,1701,1360088,1168298,1165383
2,USA Today,"2,192,098","1,674,306",-24%,1,1,2,331,449,459,1125,1360088,1168298,1165383
3,Kansas City Star,"275,747","189,283",-31%,2,0,2,99,143,147,1670,1360088,1168298,1165383
4,St. Louis Post-Dispatch,"281,198","167,199",-41%,4,3,7,215,275,282,1858,1360088,1168298,1165383
5,Newsday,"553,117","377,744",-32%,12,6,18,958,1330,1391,1194,1360088,1168298,1165383
6,Detroit Free Press,"379,304","209,652",-45%,7,5,12,382,429,445,1757,1360088,1168298,1165383
7,Arizona Republic,"466,926","293,640",-37%,5,2,7,204,270,280,2025,1360088,1168298,1165383
8,Charlotte Observer,"231,369","137,829",-40%,1,3,4,332,454,474,1869,1360088,1168298,1165383
9,Columbus Dispatch,"259,127","137,148",-47%,1,0,1,451,564,591,1691,1360088,1168298,1165383
